In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from fastai.data.transforms import get_image_files
from fastai.vision.data import SegmentationDataLoaders
from os import path

In [2]:
base_dir = "/media/nevin/Trash Games1/culane"
image_files = get_image_files(path.join(base_dir, "test/"))

In [3]:
def label_func(input):
    return f"{base_dir}/test_labels/{input.stem}.lines.txt"

print(label_func(image_files[0]))

/media/nevin/Trash Games1/culane/test_labels/00000.lines.txt


In [4]:
codes = ["lane"]

In [5]:
dls = SegmentationDataLoaders.from_label_func(
    path="/media/nevin/Trash Games1/culane",
    bs=4,
    fnames=images_files,
    label_func=label_func,
)

NameError: name 'images_files' is not defined

In [ ]:
dls[0]